# Self-Attention

Self-Attention最早出现在2017年Google翻译团队提出的论文《Attention is all you need》中，其完全摒弃了RNN（其各类变种循环神经网络）和CNN（各类基于卷积核的方法）两者的设计思想和结构方法，而仅仅采用Attention机制进行机器翻译任务，在最后的效果上取得了巨大的成功。

## self-attention的作用

为什么Self-Attention会在机器翻译领域取得巨大成功以及日后迁移到图像领域仍然表现的奔逸绝尘，在于其机制方法不同于RNN需要计算多次才能得到整个输入数据的时序特征（即拿到输入数据当前帧与前面帧或者后面帧的前后联系，在大多数情况下离当前帧数据太远的帧其时序特征相关性计算存在困难）以及CNN方法总是使用固定尺寸的滑窗内积原始数据得到当前尺寸窗口的特征而很难得到前后特征之间的内在联系。

以机器翻译任务为例，输入是一个英文句子。当模型在处理每个单词时，Self-Attention可以允许模型查看输入序列的其他位置，以助于更好的编码当前单词的线索。即Self-attention是Transformer用来将其他相关单词的“理解”用来融入当前正在处理的单词的方法，其可以捕获同一个句子中单词之间的一些句法以及语义特征。如下图展示的那样。

<center class="half">
    <img src = "../images/句法特征.png"; style="zoom:60%; transform:rotate(90deg)"/><img src = "../images/单词间的语义特征.png"; style="zoom:60%; transform:rotate(90deg)"/>
</center>

引入Self-Attention后会更容易捕获句子中长距离的相互依赖的特征，因为如果是RNN或者LSTM，需要依序列计算，对于远距离的相互依赖的特征，要经过若干时间步步骤的信息累积才能将两者联系起来，而距离越远，有效捕获的可能性越小。

但是Self-Attention在计算过程中会直接将句子中任意两个单词的联系通过一个计算步骤直接联系起来，所以远距离依赖特征之间的距离被极大缩短，有利于有效地利用这些特征。除此外，Self-Attention对于增加计算的并行性也有直接帮助作用。

## Self-Attention的计算过程

### 第一步：初始化Q、K、V
首先Self-Attention将输入的单词序列利用Embedding方法转化为嵌入向量，然后对输入的每一个embedding向量都创建三个向量，分别是：Query向量、Key向量以及Value向量。这些向量都是通过将Embedding向量乘以在训练过程中创建的三个可学习的权值矩阵而得到的。

<div align=center>

![image](../images/first.png)

在上图中，输入数据为两个单词。Thinking经过Embedding操作后转化为了高维的嵌入向量，然后分别乘以训练过程中创建的 $W^{Q}, W^{K}, W^{V}$ 三个权值矩阵后得到其对应的Query向量，Key向量和Value向量，这样可以为每个输入单词都计算出以上三个向量。

这些新向量的维度相较于原Embedding向量小。此处举例：它们的维度可以为64，而Embedding向量的维度在512维，它们不必更小，这是一种架构选择，可以使得多头注意力的计算保持不变。

### 第二步： 计算Self-attention Score

在这一步中，需要根据当前单词对输入句子中所含的每个词进行评分，这个分数决定了对输入句子其他单词的关注程度。假设我们正在计算本例中第一个单词“Thinking”的自注意力。我们需要根据这个词对输入句子的每个词进行评分。当我们在某个位置对单词进行编码时，分数决定了将多少注意力放在输入句子的其他部分上。

得分是通过将Query向量与我们正在评分的各个单词的Key向量进行点积来计算的。 因此，如果我们正在处理位置 1 中单词的自注意力，第一个分数将是q1和k1的点积。第二个分数是q1和k2的点积。

计算方法如下图所示。

<div align=center>

![image](../images/second.png)

### 第三步： 对Self-Attention Score进行缩放和归一化，得到Softmax Sorce

对上一步得到的分数进行缩放，在Google的原论文中是将其除以8（即向量维度的平方根——Q、K、V向量均为64维），这样可以让模型具有更稳定的梯度，最后再进行Softmax操作进行归一化处理。

<dvi align=center>

![img](../images/third.png)

### Softmax score乘以Value向量，求和得到Attention Value

每个Value向量（包括句段中的每一个词的Value），乘以上一步输出的Softmax score得到加权后的V1和V2，最后再将$V_{i} , i \in [1,2,3...]$相加得到Z1，这样就得到了该句段第一个单词thinking对注意力值，其也为送到前馈神经网络的向量。其他单词用相同的方法进行计算。

<center class="half">
    <img src = "../images/fourth.png"; style="zoom:70%;"/>
</center>

### Self-Attention的矩阵形式计算

![](./images/matrix.png)

## Multi-Head Attention

多头注意力机制使用不同的权值矩阵进行8次（参考论文）相同过程的计算，最后对每一个单词会得到8个不同的Z矩阵。

这样做带来了一些好处如：
- 扩展了模型关注不同位置的能力。当多头注意力模型和自注意力机制集合的时候，比如我们翻译“动物没有过马路，因为它太累了”这样的句子的时候，我们想知道“它”指的是哪个词，如果能分析出来代表动物，就很有用。
- 为Self-Attention层提供了多个“表示子空间”。对于多头注意力，我们不仅有一个，而且还有多组Query/Key/Value权重矩阵，这些权重矩阵集合中的每一个都是随机初始化的。然后，在训练之后，每组用于将输入Embedding投影到不同的表示子空间中。多个head学习到的Attention侧重点可能略有不同，这样给了模型更大的容量。

当然最后对得到的8个Z矩阵，需要对其进行拼接操作，再将它们乘以一个额外的权重矩阵 $W^{O}$ ，得到最后特征加权后的Z矩阵。

其计算过程和多头注意力机制的总体计算流程如下图所示。

<center class="half">
    <img src = "../images/multihead_Zmatrix.png"; style="zoom:70%;"/><img src = "../images/multihead_process.png"; style="zoom:70%;"/>
</center>

## 总结

Self-Attention就是Q、K、V均为同一个输入向量映射而来的Encoder-Decoder Attention，它可以无视词之间的距离直接计算依赖关系，能够学习一个句子的内部结构，实现也较为简单并且可以并行计算。

Multi-Head Attention同时计算多个Attention，并最终得到合并结果，通过计算多次来捕获不同子空间上的相关信息。